# Geographic Data With GeoPandas

We have previously used the Pandas library to load and explore data.  For geographic data there is an extension to Pandas, called **GeoPandas**, which knows about geographic information, and how to draw maps of it.

In this worksheet you will learn to read the most common types of geographic data, and display maps with python

There are several other good libraries in python for handling different aspects of geographic data.

## Using this notebook

- You should already have completed [the numpy notebook](1_Numpy.ipynb), [the matplotlib notebook](2_Matplotlib.ipynb) , and [the pandas notebook](3_Pandas.ipynb) before trying this one.  The layout is the same.


- **This notebook is graded: there are 11 exercises**


- **Make sure to execute every cell or the ones below may not work**


- **Note: This notebook is a fair bit more difficult than previous ones!  It is mainly intended to give you some examples you can use in the intensive hackathon. Don't worry if you find it harder or don't get through it all** 


Before you start, we need to run this cell to install the latest version of geopandas:

In [ ]:
!conda install -y "geopandas>=0.9.0"

In [ ]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

---

# 1. Shape Files for Geographic Data

Various quantities can be related to a map - individual points, lines, such as borders, or entire regions, such as political or social divisions.  These can be stored in a *shape file* - a file format that encodes points, lines and polygons on the Earth's surface.  

In Pandas we used the `DataFrame` object to store tables of data. In GeoPandas, the equivalent is called a `GeoDataFrame`.  It is a DataFrame with special extra columns and attributes to store geographic information.

GeoPandas has a function, `read_file`, specifically to read a shape file into a `GeoDataFrame`.  We will use that, and then the `.head()` method which shows the top few rows of the data:

In [ ]:
regions = gpd.read_file("data/Local_Authority_Districts_(December_2018)_Boundaries_UK_BFC.zip")
regions.head()

You can see some columns for the name and various coordinates, and also the geometry column, which contains the more details shape information.  We don't have to delve into the details - GeoPandas can just use this information for us.

---

Anything you can do with a regular Pandas DataFrame you can also do with this GeoDataFrame.

## <font color='blue'>Exercise 1</font>  

Sort this data frame by the column that represents the area of the local authorities, and find the largest and smalles ones.


In [ ]:
# Space for your workings for the exercise


---


# 2. Plotting Maps


Now let's go on exploring our data.  GeoPandas lets us visualize all the borders as lines, by making an empty figure with an axis in it, and then
passing that to the `regions.plot` method with some choices for the colors and lines.

In [ ]:
ax = regions.plot(facecolor='white', edgecolor='black', linewidth=0.1, figsize=(5,8))

That looks familiar.  

---

## <font color='blue'>Exercise 2</font>  


Make a version of this plot with:
- blue border lines
- red filled-in authorities
- sensibly labelled x and y axes

Hint: The `plot` method returns an axis object which has methods `set_xlabel` and `set_ylabel`.


In [ ]:
# Space for your workings for the exercise


---


# 3. Combining Data

Now let's read a second data file that we would like to use in our mapping.  

We will need to be able to match it in some
way to our regions information, so we should use something that is also organise by local authority.  In particular, let's use some demographic data about age distributions in the UK.  This is a regular comma-separated values file (CSV) that we will read with standard Pandas:

In [ ]:
ages = pd.read_csv("data/age_distributions_2018.csv.gz")
ages.head()

This looks sensible.  It has different columns for each possible age, and Male `M`, Female `F` and All `A` columns.

---

Let's plot something simple and non-geographic: a histogram of the populations of local authorities.  We can use the `hist` method of the data frames we just loaded (this is specific variant of the `plot` method we used in the last exercise):

In [ ]:
axes = ages.hist('ATotal', bins=50)

Hmm.  Not great.  There's one row with about 65 million people in, and all the rest much closer to zero.

---

Here's where our domain-specific knowledge comes in: we recognise that number as the total UK population.  There must be a row for the whole UK.

## <font color='blue'>Exercise 3</font>  


Replaces `ages` variable with a new version that doesn't have this row in.  There are a few ways you could do this - you could figure out which row it is and use `ages.drop`, or you could `query` all rows with ATotal less than (e.g.) 50 million,.


In [ ]:
# Space for your workings for the exercise


---

Now let's merge our region data with our age distribution data.  We use the GeoPandas `merge` method, with the names of the two columns we want to match together.  These are called the left and right columns, because we imagine one table on the left-hand side and matching up row by row with another table on the right-hand side:

In [ ]:
match = regions.merge(ages, left_on='lad18nm', right_on='Local Authority')
match.head()

It looks like something has worked and we've managed to match the two tables.  

Let's check by plotting the borders again, from this new matched table:

In [ ]:
match.plot(figsize=(5,8))

Wales is missing!  

That must mean that when we did the match the names of places in Wales were not paired up. Let's try to figure out why - you might have a guess already.

---

We can speed up our search a lot using python `sets`.  A set is a structure of data that makes it very quick to check whether a particular object is in the set (if we were using an array or a list we would have to search through all the data every single time, which is slow.  Sets use a lookup table to make it faster).

In [ ]:
# Make a set of each set of names
region_names = set(regions['lad18nm'])
ages_names = set(ages['Local Authority'])

## <font color='blue'>Exercise 4</font>  


a) Write two python loops, one through each of these two sets.  In each case, check if the loop variable is contained in the *other* set, and print out if it is not.

Hint: the python syntax `x in y` checks if the item `x` is in the set `y` and evaluates as `True` or `False`.

b) Write down an explanation of the problem with the matching that this reveals.


---

In [ ]:
# Space for your workings on this exercise part (a)


In [ ]:
# Space for your explanation of the problem part (b)
# write your answer as a comment


--- 

# 4. Choropleths

Your diagnosis now lets us fix the problem, by making a new column in the `ages` table with a "fixed" name.  
This kind of data cleaning is the bulk of real-world data analysis!

We will use a python *list comprehension* to make the new column values in a list, and then we can easily create the new column.  For convenience we can give it the same name as the regions table, so that the merge command is even simpler:

In [ ]:
# This makes a list for each local authority name, consisting of the item, split my the '/' character,
# and then with the extra space stripped off
new_column = [name.split('/')[0].strip() for name in ages['Local Authority']]

# Add our new column
ages['lad18nm'] = new_column

# Make a new match
match2 = regions.merge(ages, on='lad18nm')

--- 

Now we have hopefully fixed Wales we can use GeoPandas to plot a specific colour for a specific column.  This kind of geographic map of a numeric quantity is called a *choropleth*.

Let's look at a choropleth of the total population of each local authority.  Play around with this method and see what options you have.

In [ ]:
match2.plot('ATotal', legend=True, figsize=(5,8))

---

Now lets explore the gender ratio in different parts of the UK.

## <font color='blue'>Exercise 5</font>  


Make a new column in the `match2` table which contains the ratio of the total number of men to the total number of women, and then make a map of that ratio


In [ ]:
# Space for your workings for the exercise 


--- 

# 5. Individual Locations

Now lets look at some point-like data (i.e. individual locations, instead of lines or regions), and learn how to convert it to the kind of maps above.

We will load the location of every post office in the United Kingdom from a new CSV file:

In [ ]:
post_offices = pd.read_csv("data/pol-branch-listings-2020-02-20.csv.gz")
post_offices.head(10)

You can ignore any warning that appears; it doesn't matter here. What does matter is that each post offices appears in the list multiple times, once for each day of the week that it is open.  We should fix that.

---

## <font color='blue'>Exercise 6</font>  

Replace the `post_offices` variable with one with the duplicates for each location removed.
Hint: Consult the documentation for the `drop_duplicates` method of pandas data frames, and figure out which column to use.


In [ ]:
# Space for your workings for exercise 7


--- 

The post_offices variable is still a Pandas data frame; it is not a GeoPandas object with full coordinates yet.  But it is in latitude/longitude coordinates, so we can plot those:


## <font color='blue'>Exercise 7</font>  

Make a basic Pandas scatter plot of the longitude vs latitude in the `post_offices` table.

Hint: Set the keyword `s=0.1` to make the size of each point smaller so it is easier to see them all.

In [ ]:
# Space for your workings for this exercise


---

To convert to a full GeoDataFrame we have to first make a geometry object that describes the location.  GeoPandas has a function to do this for us, `points_from_xy`.  We can use this to make a full GeoPandas GeoDataFrame.

We also (optionally; this will be useful later) tell GeoPandas what *coordinate reference system* (CRS) we are working in. The most common latitude and longitude system in geo data files is called `WGS84`, sometimes also called by the code name `epsg:4326`.  If you just have a general latitude and longitude then this is the system it will almost certainly use.

In [ ]:
geometry = gpd.points_from_xy(post_offices['BRANCH_LONGITUDE'], post_offices['BRANCH_LATITUDE'], crs='epsg:4326')
post_offices = gpd.GeoDataFrame(post_offices, geometry=geometry)

# 6. Coordinate Systems

Now we can plot our local authorities on the same plot as our post offices.  We have to set the face color to "none" so that it doesn't draw over the post office points when it draws the local authorities.

In [ ]:
ax = post_offices.plot(markersize=0.1, color='red', figsize=(6, 8))
regions.plot(facecolor='none', edgecolor='black', linewidth=0.1, ax=ax)

This doesn't look right.  If you look closely you will see that the red dots for the post offices are all down at coordinates (0, 0).  That gives us a clue as to what has happened - all our maps of the local authorities are in meters, with the zero point at the southern and western corner of the UK, but our post office plot was in degrees latitude and longitude.

This is a problem with coordinate systems.  We need to convert either one to match the other.

## <font color='blue'>Exercise 8</font>  

Use the `post_offices.to_crs` method to convert the post-office data to a new variable `post_offices2` with the same CRS as the regions data.

You will need to figure out how to get the CRS information object from the `regions` table.

Hint: In the cell below type `regions.` and then press the Tab key on your keyboard.  This will give you a list you can scroll through of all the attributes of the regions variable.  Look for one that could be the CRS.

In [ ]:
# Space for your workings on this exercise


---

If that has worked you will now be able to make a much more sensible looking plot:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 8))
post_offices2.plot(markersize=0.1, color='red', ax=ax)
regions.plot(facecolor='none', edgecolor='black', linewidth=0.1, ax=ax)

---

# 7. Spatial Joins

Geographical relationships in GeoPandas, like checking whether a point is inside a region, or whether two regions intersect, are done using a *spatial join*, using the `sjoin` function.  It makes a new table, with the combined columns of the two input (points and regions).  The rows of the table are all the cases where there is a match.

The first few columns will be the same for each row in what is printed below

(In this case because local authorities don't overlap, each post office will only be in a single region

In [ ]:
match_table = gpd.sjoin(regions, post_offices2, how="inner", op='contains')
match_table.head(5)

---

Now we have made this matched table we can make a new column in our regions data, and fill it with the
number of post offices in that region.  There are a few ways we could do that.  Here, we will loop through the different regions and find all the post offices in it one by one.

In [ ]:
match2['post_office_count'] = np.zeros(len(regions))
for i in regions.index:
    name = match2.at[i, "lad18nm"]
    # This is a series of True/False values indicating whether reach row in the match
    # table as the same name as our curent region
    matched_rows = (match_table['lad18nm'] == name)
    # In python True counts as 1 and False as 0 for the purposes
    # of arithmetic.  So if we sum up all the values then that is the
    # True in the series.
    matched_count = matched_rows.sum()
    match2.at[i, "post_office_count"] = matched_count
    

## <font color='blue'>Exercise 9</font>  

Make a new column in the `match2` table showing the number of people per post office in the authority, and display a map of it.

In [ ]:
# Space for your workings on this exercise


# 8. Geocoding

We frequently don't have a latitude and longitude in our data sets, but some other location information, such as an address, postcode, or place name.

The process of turning information like this into a latitude & longitude that we can map is called *geocoding*.  To perform general geocoding we have to use one of various web services.  These services usually either limit your usage (e.g. 100 postcodes per day) or cost money, though not typically very much.

## <font color='blue'>Exercise 10</font>  

Use the GeoPandas function `gpd.tools.geocode` to make a variable called `d` of the latitude and longitude of the Edinburgh Futures Institute, 


In [ ]:
# Space for your workings on this exercise


--- 

Now we have this location we can plot it on our map.  We have to convert coordinates again first:

In [ ]:
d2 = d.to_crs(regions.crs)
ax = regions.plot(figsize=(5,8))
d2.plot(ax=ax, color='red', marker='x', markersize=200)
ax.set_xlim(2e5, 5e5)
ax.set_ylim(6e5, 8e5)

For UK postcode data in particular we can instead download the full postcode data base, subject to a licenses.

If you like, and the file is not too large, have a go reading the file `data/NSPL_NOV_2020_UK.csv.gz` and finding the latitude and longitude of the Edinburgh Furniture Initiative at EH7 4HF


---

# 9. Wrapping up

You're now ready for the hackathon!

## <font color='blue'>Exercise 11</font>  

Explore & play.  Do something interesting with the data you've loaded so far; either plot something, or look at the methods of geopandas data sets and see what you can do.

In [ ]:
# Space for your workings on this exercise
